# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState

RANDOM_STATE = 12345

In [2]:
data_0 = pd.read_csv("/datasets/geo_data_0.csv")
data_1 = pd.read_csv("/datasets/geo_data_1.csv")
data_2 = pd.read_csv("/datasets/geo_data_2.csv")

In [3]:
display(data_0.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [4]:
display(data_1.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [5]:
display(data_2.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


None

In [6]:
display(data_0.head())
display(data_1.head())
display(data_2.head())

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [7]:
display(data_0.duplicated().sum())
display(data_1.duplicated().sum())
display(data_2.duplicated().sum())

0

0

0

**Вывод:**

Как видно из анализа, данных больше чем предполагалось. Было заявлено 10.000, а на самом деле 100.000
Однако, данные (тип данных) соответствуют описанию по своему наполнению, не имеют пропусков и дубликатов. Можно приступать к моделированию. 

## Обучение и проверка модели

Так как нам нужно произвести разделение данных на тренировочную и валидационую выборку для всех данных, а позже построить модель, то лучше всего создать функцию и прогнать через нее все три группы данных. Результатом будет значение RMSE и средний запас предсказаного сырья.

In [8]:
def predict(data):
    features = data.drop(["product","id"], axis = 1)
    target = data["product"]
    
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=0.25, random_state=RANDOM_STATE)
    
    model = LinearRegression()
    model.fit(features_train,target_train)
    predictions = model.predict(features_valid)
    
    RMSE = (mean_squared_error(target_valid,predictions)**0.5)
    print("RMSE:",round(RMSE,2))
    print("Средний запас предсказаного сырья",round(predictions.mean(),2),"тыс. баррелей")
    
    return target_valid, predictions

In [9]:
data_0_target_valid, data_0_predictions = predict(data_0)

RMSE: 37.58
Средний запас предсказаного сырья 92.59 тыс. баррелей


In [10]:
data_1_target_valid, data_1_predictions = predict(data_1)

RMSE: 0.89
Средний запас предсказаного сырья 68.73 тыс. баррелей


In [11]:
data_2_target_valid, data_2_predictions = predict(data_2)

RMSE: 40.03
Средний запас предсказаного сырья 94.97 тыс. баррелей


**Вывод:**

Самый низкий показатель запаса по сырью имеет 1 регион (68.7 тыс. баррелей), однако значение RMSE тоже низкое и составляет всего 0.89, что говорит о хорошо построенной модели. 

Остальные регионы 0 и 2, имеют почти на 35% больше сырья, 92.59 и 94.97 тыс. баррелей соответственно. Однако, точность моделей хромает со значениями RMSE 37.58 для 0 группы и 40.03 для 2 группы. Поэтому, не стоит брать высокое значение по сырью в серьоз. 

## Подготовка к расчёту прибыли

Проведем расчет нужного количества сырья для прибольной скважены:

In [12]:
budget = 10*10**9
best_wells = 200
sample_wells = 500
unit_income = 450000
min_product_amount = budget/(best_wells*unit_income)

In [13]:
print("Минимально требованный объем с одной скважины:",round(min_product_amount,2),"тыс. баррелей")

Минимально требованный объем с одной скважины: 111.11 тыс. баррелей


**Вывод:**

Минимальный объем сырья для прибыльной скважины равен 111.11 тыс. баррелей. Данное значение выше, чем все средние объёмы сырья полученные в разделе 2.

## Расчёт прибыли и рисков 

### Расчет прибыли

Для подсчета прибыли для 200 (best_wells) лучших скважен из рандомных 500 (sample_wells), напишем функцию income(). 
Так же, для будущего bootstrap'а, добавим возможноть обращения к функции без вызова печати результатов за счет переменной show. 

In [14]:
def income(target,predictions,state,show=True):
    
    random_500_predictions = pd.Series(predictions).sample(n=sample_wells,replace=False,random_state=state)
    top_200_predictions = random_500_predictions.sort_values(ascending=False)[:best_wells]
    
    top_200_predictions_mean = top_200_predictions.mean()
    top_200_target = target.iloc[top_200_predictions.index]
    
    product_sum = top_200_target.sum()
    income = product_sum * unit_income - budget
    
    if show:
        print("Средний запас сырья среди топ 200 скважен:",round(top_200_predictions_mean,2),"тыс. баррелей")
        print("Общий объем сырья:",round(product_sum,2),"тыс. баррелей")
        print("Прибыль:",round(income/10**6,2),"млн.")
    else:
        return income

In [15]:
income(data_0_target_valid,data_0_predictions,RANDOM_STATE)

Средний запас сырья среди топ 200 скважен: 116.73 тыс. баррелей
Общий объем сырья: 23731.26 тыс. баррелей
Прибыль: 679.07 млн.


In [16]:
income(data_1_target_valid,data_1_predictions,RANDOM_STATE)

Средний запас сырья среди топ 200 скважен: 119.77 тыс. баррелей
Общий объем сырья: 23954.4 тыс. баррелей
Прибыль: 779.48 млн.


In [17]:
income(data_2_target_valid,data_2_predictions,RANDOM_STATE)

Средний запас сырья среди топ 200 скважен: 114.89 тыс. баррелей
Общий объем сырья: 23199.98 тыс. баррелей
Прибыль: 439.99 млн.


### Расчет рисков

Для подсчета доверительного интервала и риска применим технику Bootstrap в функции risk():

In [18]:
def risk(target,predictions):
    
    bootstraps_samples = 1000
    alpha = 0.05 
    income_list = []
    state = np.random.RandomState(RANDOM_STATE)
    
    for i in range(bootstraps_samples):
        new_income = income(target,predictions,state,show=False)
        income_list.append(new_income)
        
    income_list = pd.Series(income_list)
    left = income_list.quantile(alpha/2)
    right = income_list.quantile(1 - (alpha/2))
    
    count = 0 
    for i in income_list: 
        if i < 0 :
            count += 1

    risk = count/bootstraps_samples
    
    print("Средняя прибыль:",round(income_list.mean()/10**6,2),"млн.")
    print("Доверительный интервал:",round(left/10**6,2),"-",round(right/10**6,2),"млн.")
    print("Риск:",round(risk*100,2),"%")

In [19]:
risk(data_0_target_valid,data_0_predictions)

Средняя прибыль: 380.71 млн.
Доверительный интервал: -126.95 - 879.61 млн.
Риск: 7.2 %


In [20]:
risk(data_1_target_valid,data_1_predictions)

Средняя прибыль: 448.23 млн.
Доверительный интервал: 70.9 - 892.99 млн.
Риск: 1.4 %


In [21]:
risk(data_2_target_valid,data_2_predictions)

Средняя прибыль: 402.8 млн.
Доверительный интервал: -143.66 - 963.03 млн.
Риск: 7.1 %


**Вывод:**

Если брать 200 лучших скважен из 500 рандомных, то показатель среднего объема сырья уже будет составлять 116.73, 119.77 и 114.89 для групп 0, 1 и 2 соответственно. Данный показатель уже выше минимального нужного сырья для без убыточности из раздела 3. 

После проверки на риск, можно сделать вывод, что регион 1 лучше всего подходит для разработки. В этом регионе показал лучший интервал с без убыточными значения в 70.9 - 892.99 млн., с риском в 1.4%. Тогда как регионы 0 и 2 имеют высокие риски (7.2 и 7.1 соответственно) и в доверительном интервале присутствует убыточное значение. 

## Финальный Вывод

Отвечая на главный вопрос по выбору региона для разработки, который принесет наибольшую прибыль, то однозначным выбором является **регион 1**. По технике Bootstrap, данный регион имеет самый высокий показатель средний прибыли среди всех регионов, с наилучшим доверительным интервалом в 70.9 - 892.99 млн. и риском 1.4%. Так же, модель по данному региону показала лучшую точность с RMSE = 0.89, что говорит о хорошей возможности данной модели для определения региона. 